In [6]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
import csv
import glob

sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/buslead_verified.db')
sqlite_connection = sqlite_engine.connect()

grade_file_path = '/Users/saisun/ProjectCode/uqx_tools/micromastersreports/data/'
grade_file_postfix = "_grade_report_*.csv"
profile_file_postfix = "_student_profile_info_*.csv"


#micromasters_courses = {
#    "buslead1x": {"title":"Name 1", "instance_codes":['UQx_BUSLEAD1x_3T2017', 'UQx_BUSLEAD1x_2T2018']},
#    "buslead2x": {"title":"Name 2", "instance_codes":['UQx_BUSLEAD2x_1T2018', 'UQx_BUSLEAD2x_2T2018']},
#    "buslead3x": {"title":"Name 3", "instance_codes":['UQx_BUSLEAD3x_1T2018']},
#    "buslead4x": {"title":"Name 4", "instance_codes":['UQx_BUSLEAD4x_2T2018']},
#    "buslead5x": {"title":"Name 5", "instance_codes":['UQx_BUSLEAD5x_3T2018a']}    
#}

micromasters_courses = {
    "buslead1x": {"title": "UQx_BUSLEAD1x", "instance_codes": ['UQx_BUSLEAD1x_*'], 'grade': 0.65},
    "buslead2x": {"title": "UQx_BUSLEAD2x", "instance_codes": ['UQx_BUSLEAD2x_*'], 'grade': 0.65},
    "buslead3x": {"title": "UQx_BUSLEAD3x", "instance_codes": ['UQx_BUSLEAD3x_*'], 'grade': 0.65},
    "buslead4x": {"title": "UQx_BUSLEAD4x", "instance_codes": ['UQx_BUSLEAD4x_*'], 'grade': 0.65},
    "buslead5x": {"title": "UQx_BUSLEAD5x", "instance_codes": ['UQx_BUSLEAD5x_*'], 'grade': 0.65},
}


def db_refresh():
    sqlite_connection.execute("DELETE FROM learners")
    sqlite_connection.execute("DELETE FROM course_instancedetails")
    sqlite_connection.execute("DELETE FROM buslead_coursecompletion")
    
def db_insert (table, row):

    keys = row.keys();
    sql = 'INSERT INTO ' + table + ' ('
    sql = sql + ', '.join(keys)
    sql = sql + ') VALUES ('
    sql = sql + ', '.join([ ('"' + str(row[key]) + '"') for key in keys])
    sql = sql + ')'
    id = sqlite_connection.execute(text(sql));

    return id

def import_gradereport_fromcsv(file_to_import, course, course_prefix):
    mydict = None
    
    for file_name in glob.glob(file_to_import):  
        course_instance_start = int(file_name.find(course_prefix))
        course_instance_end = file_name.find('_grade_report')
        #course_instance_end = int(course_instance_start + len(course_prefix) + 7)
        #print(file_name, course_instance_start, course_instance_end)
        course_instance = file_name[course_instance_start:course_instance_end]
        print(course_instance)
        with open(file_name, mode='r', encoding='utf-8-sig') as infile:
            reader = csv.DictReader(infile)
            for row in reader:
                student_id = int(row['Student ID'])
                email = row['Email']
                email = email.replace("'", "")
                username = row['Username']   
                import_learner(student_id, email, username)
                #insert course instance details
                grade = row['Grade']
                enrollment_track = row['Enrollment Track']
                verification_status = row['Verification Status']
                certificate_eligible = row['Certificate Eligible']
                certificate_delivered = row['Certificate Delivered']
                row = {
                    'learner_id': student_id,
                    'grade': grade,
                    'enrollment_track': enrollment_track,
                    'verification_status': verification_status,
                    'certificate_eligible': certificate_eligible,
                    'certificate_delivered': certificate_delivered,
                    'course': course,
                    'course_instance': course_instance
                }
                db_insert('course_instancedetails', row)
            

def import_learner(student_id, email, username):
    sql = "SELECT COUNT(*) FROM learners where student_id=%d;" % (student_id)
    result = sqlite_connection.execute(sql);
    no_students = 0
    for row in result:
        no_students = row[0]
    if no_students == 0:
        row = {'student_id': student_id, 'email':email, 'username':username}
        db_insert('learners', row)

def export_micromasters_completionreport():
    print("export_micromasters_completionreport")
    sqlite_connection.execute("DELETE FROM buslead_coursecompletion");
    sql = "SELECT * FROM learners;"
    result = sqlite_connection.execute(sql);
    for row in result:
        #print(row)
        learner_id = row[1]  
        username = row[4]
        name = row[3]
        email = row[2]
        learner_dict = {'student_id':learner_id, 'username':username,'name':name, 'email':email}
        no_completed_courses = 0
        #print(learner_id, username)
        for course in micromasters_courses:
            course_dict = micromasters_courses[course]
            grade_certificate = course_dict['grade']
            #print(grade_certificate)
            #print(course_dict['instance_codes'])
            learner_dict[course + "_" + 'verified'] = "-"
            learner_dict[course + "_" + 'completed'] = "-"
            courseinstance_sql = "SELECT * FROM course_instancedetails WHERE course='%s' AND learner_id=%d;" % (course, learner_id)
            courseinstance_result = sqlite_connection.execute(courseinstance_sql);
            verified_completed = False
            for courseinstance_row in courseinstance_result:
                #print(courseinstance_row)
                enrollment_track = courseinstance_row[3]
                certificate_eligible = courseinstance_row[8]
                grade = courseinstance_row[2]
                if grade >= grade_certificate:
                    certificate_eligible = 'Y'
                if verified_completed == False:
                    learner_dict[course + "_" + 'verified'] = enrollment_track
                    learner_dict[course + "_" + 'completed'] = certificate_eligible
                    if ((certificate_eligible == 'Y') and (enrollment_track == 'verified')):
                        no_completed_courses += 1
                        verified_completed = True
                        #print(username, certificate_eligible, enrollment_track, no_completed_courses)
                        break
        #print(learner_dict)
        learner_dict['no_completed_courses'] = no_completed_courses
        #print(learner_dict)
        db_insert('buslead_coursecompletion', learner_dict)

def micromasters_completionreport_count_auditpassed():
    print("micromasters_completionreport_count_auditpassed")
    sql = "SELECT * FROM buslead_coursecompletion;"
    result = sqlite_connection.execute(sql);
    for row in result:
        update_id = int(row[0])
        learner_dict = {'buslead1x_verified':row[3], 'buslead1x_completed':row[4],
                        'buslead2x_verified':row[5], 'buslead2x_completed':row[6],
                        'buslead3x_verified':row[7], 'buslead3x_completed':row[8],
                        'buslead4x_verified':row[9], 'buslead4x_completed':row[10]
                       }
        no_auditedpassed_courses = 0
        if (learner_dict['buslead1x_verified'] == 'audit') and (learner_dict['buslead1x_completed'] == 'Y'):
            no_auditedpassed_courses +=1
        if (learner_dict['buslead2x_verified'] == 'audit') and (learner_dict['buslead2x_completed'] == 'Y'):
            no_auditedpassed_courses +=1
        if (learner_dict['buslead3x_verified'] == 'audit') and (learner_dict['buslead3x_completed'] == 'Y'):
            no_auditedpassed_courses +=1
        if (learner_dict['buslead4x_verified'] == 'audit') and (learner_dict['buslead4x_completed'] == 'Y'):
            no_auditedpassed_courses +=1
        
        updatesql = "UPDATE buslead_coursecompletion SET no_audited_passed=%d WHERE id=%d;" % (no_auditedpassed_courses, update_id)
        sqlite_connection.execute(updatesql);
        
def update_learnername_fromcsv(profile_file_to_import):
    
    for file_name in glob.glob(profile_file_to_import):    
        with open(file_name, mode='r', encoding='utf-8-sig') as infile:
            reader = csv.DictReader(infile)
            for row in reader:
                #print(row)
                name = row['name']
                name = name.replace('\'', '')
                name = name.replace("\"", "")
                                    
                #name = name.replace('"Ahmad Ezzat"', 'Ahmad Ezzat')
                #name = name.replace("Moh'd", "Mohd")
                #name = name.replace('"Mohd Khair"', 'Mohd Khair')

                student_id = int(row['id'])
                country = row['country']
                #print(country)
                updatesql = 'UPDATE learners SET name="%s" WHERE student_id=%d;' % (name, student_id)
                #print(updatesql)
                sqlite_connection.execute(updatesql);

                updatesql = "UPDATE buslead_coursecompletion SET country='%s' WHERE student_id=%d;" % (country, student_id)
                sqlite_connection.execute(updatesql);

db_refresh()
for course in micromasters_courses:
    course_dict = micromasters_courses[course]
    print(course_dict['title'])
    file_to_import = grade_file_path + course_dict['title'] + '_*' + grade_file_postfix
    #print (file_to_import)
    import_gradereport_fromcsv(file_to_import, course, course_dict['title'])
    profile_file_to_import = grade_file_path + course_dict['title'] + '_*' + profile_file_postfix
    update_learnername_fromcsv(profile_file_to_import)

    
#    for course_instance in course_dict['instance_codes']:
#        print(course_instance)
#        file_to_import = grade_file_path + course_instance + grade_file_postfix
#        print(file_to_import)
#        import_gradereport_fromcsv(file_to_import, course, course_instance)
#        profile_file_to_import = grade_file_path + course_instance + profile_file_postfix
#        update_learnername_fromcsv(profile_file_to_import)


export_micromasters_completionreport()
micromasters_completionreport_count_auditpassed()
print('The End.')


UQx_BUSLEAD1x
UQx_BUSLEAD1x_3T2017
UQx_BUSLEAD1x_2T2018
UQx_BUSLEAD1x_1T2019
UQx_BUSLEAD2x
UQx_BUSLEAD2x_1T2019
UQx_BUSLEAD2x_2T2018
UQx_BUSLEAD2x_1T2018
UQx_BUSLEAD3x
UQx_BUSLEAD3x_1T2019
UQx_BUSLEAD3x_1T2018
UQx_BUSLEAD4x
UQx_BUSLEAD4x_1T2019
UQx_BUSLEAD4x_2T2018
UQx_BUSLEAD5x
UQx_BUSLEAD5x_3T2018a
UQx_BUSLEAD5x_1T2019
export_micromasters_completionreport
micromasters_completionreport_count_auditpassed
The End.
